# Deep Linking: Adding deep links and persistent sessions

## Basics

If you have a widget, it's possible to store its current value by linking it through the browser URL.

```python
import panel as pn

slider = pn.widgets.FloatSlider(name='Slider', start=0, end=10)

if pn.state.location:
    pn.state.location.sync(slider, {'value': 'slider_value'})  # rename value -> slider_value in URL

slider.servable()
```

### Exercise 1

Click [here](http://localhost:8888/panel-preview/render/basic_widget.py) to try it out!

1. Notice how the browser's URL updates as you update the slider.
2. Now, try setting a custom `slider_value` in the URL. Upon entering, the slider widget's value should match the custom value.
3. Next, copy this URL to a new browser tab. Again, you should see the slider widget's value set to match the custom value.

![Exercise 1 Demo](assets/basic_widget.gif)

### Take-away

If you see something interesting, you can persist the state of the widget through the browser URL and share it with your colleagues!

## Accessing the location

If you want to do something with the location, you access it programmatically.

```python
import panel as pn

price = 88
promo_code = pn.state.session_args.get('promo_code', None)
price_to_buy = pn.widgets.StaticText(value=f"Cost: $~~{price}~~Now ${new_price}")
layout = 

if promo_code and promo_code.endswith("OFF"):
    discount = float(promo_code.replace("OFF", "")) / 100
    new_price = price * discount
elif promo_code:
    
    

if pn.state.location:
    pn.state.location.sync(text_input, {'value': 'input_value'})

slider.servable()
```

## Parameterized Class

## Unsyncing for Multi-Page Apps

## Real-world Example - Stocks Explorer